# Task 2 Application of Knowledge Graph

# Question 3: Constructing a Knowledge Graph

Input: ALL files under the KnowledgeGraph folder

Description: 

1. To construct a knowledge graph, you will need to use Python and Neo4j database. 
2. To create the knowledge graph, you should install the Neo4j graph database on your computer and use Python to connect to it. 
3. Then, you can construct the knowledge graph based on the input files. 
4. Please search for relevant content on your own. In the knowledge graph you build, the node type is "company," and there are six types of edges: "compete," "cooperate," "dispute," "invest," "same_industry," and "supply." Edges can be directed, meaning from S to P, or undirected (bidirectional).

Submission (10 marks): You should document the process, and plot the knowledge graph in your report.pdf file. For detailed requirements, please refer to Appendix 1.

### 3.1-2 Python and Neo4j database initialisation
1. import packages and credentials

In [2]:
# %pip install neo4j
# %pip install py2neo
from py2neo import Graph, Subgraph, Node, Relationship
import neo4j
import pandas as pd
from neo4j import GraphDatabase


In [56]:
# credentials
uri = "bolt://localhost:7687"
username = "neo4j"  # Replace with your Neo4j username
password = "mining5002"
uri2 = 'http://localhost:7474'
driver = GraphDatabase.driver(uri, auth=(username, password))
try: 
    graph = Graph(uri, auth = (username, password))
    print('successful')
except Exception as e:
    print(f'error: {e}')

successful


2. data import

In [3]:

# Read data from CSV files
df_nodes = pd.read_csv("data/KnowledgeGraph/hidy.nodes.company.csv")
df_compete = pd.read_csv("data/KnowledgeGraph/hidy.relationships.compete.csv")
df_cooperate = pd.read_csv("data/KnowledgeGraph/hidy.relationships.cooperate.csv")
df_dispute = pd.read_csv("data/KnowledgeGraph/hidy.relationships.dispute.csv")
df_invest = pd.read_csv("data/KnowledgeGraph/hidy.relationships.invest.csv")
df_same_industry = pd.read_csv("data/KnowledgeGraph/hidy.relationships.same_industry.csv")
df_supply = pd.read_csv("data/KnowledgeGraph/hidy.relationships.supply.csv")

In [33]:
print("Head of df_nodes:")
print(df_nodes.head(3))

print("\nHead of df_compete:")
print(df_compete.head(3))

print("\nHead of df_cooperate:")
print(df_cooperate.head(3))

print("\nHead of df_dispute:")
print(df_dispute.head(3))

print("\nHead of df_invest:")
print(df_invest.head(3))

print("\nHead of df_same_industry:")
print(df_same_industry.head(3))

print("\nHead of df_supply:")
print(df_supply.head(3))

Head of df_nodes:
   :ID company_name       code   :LABEL
0    0         东诚药业  002675.SZ  company
1    1         大庆华科  000985.SZ  company
2    2         恒辉安防  300952.SZ  company

Head of df_compete:
   :START_ID  :END_ID    :TYPE             time
0       2082     2287  compete   2020/4/9 13:33
1       3348      272  compete  2018/10/29 7:16
2       1431      707  compete   2021/5/17 8:01

Head of df_cooperate:
   :START_ID  :END_ID      :TYPE             time
0       2197      245  cooperate  2019/9/17 12:00
1       1165      756  cooperate   2020/8/2 17:10
2       2082     2899  cooperate   2019/9/2 17:42

Head of df_dispute:
   :START_ID  :END_ID    :TYPE              time
0       1605     1866  dispute  2020/12/17 16:55
1       1299     1136  dispute   2022/2/22 17:55
2       2508     3950  dispute    2022/1/19 8:11

Head of df_invest:
   :START_ID  :END_ID   :TYPE
0       1165     1478  invest
1        469     3116  invest
2       3566     2272  invest

Head of df_same_industry:
  

3. writing sessions, creating nodes and relationships for all files

In [60]:
# Function to create nodes
def create_nodes(session, df):
    for index, row in df.iterrows():
        query = (
            f"CREATE (n:company {{ID: {row[':ID']}, company_name: '{row['company_name']}', code: '{row['code']}'}})"
        )
        session.run(query)

# create relationship
def create_relationships(session, df, relationship_type):
    for index, row in df.iterrows():
        query = (
            f"MATCH (a:company {{ID: {row[':START_ID']}}})"
            f"MATCH (b:company {{ID: {row[':END_ID']}}})"
        )
        if 'time' in row:  # Check if 'time' column exists in the dataframe
            query += f"CREATE (a)-[:{relationship_type} {{time: '{row['time']}'}}]->(b)"
        else:
            query += f"CREATE (a)-[:{relationship_type}]->(b)"
        session.run(query)


In [61]:
with driver.session() as session:
    create_nodes(session, df_nodes)

# Create relationships for each dataframe
def create_all_relationships():
    with driver.session() as session:
        create_relationships(session, df_compete, "compete")
        create_relationships(session, df_cooperate, "cooperate")
        create_relationships(session, df_dispute, "dispute")
        create_relationships(session, df_invest, "invest")
        create_relationships(session, df_same_industry, "same_industry")
        create_relationships(session, df_supply, "supply")

# Call the function to create all relationships
create_all_relationships()

# Close the driver
driver.close()

### 3.3-4 graph database query 

In [62]:
# Function to execute Cypher query
def execute_query(query):
    with driver.session(database="neo4j") as session:
        result = session.run(query)
        return result

In [63]:
## Merge 'compete' and 'cooperate', 'invest', 'supply' relationships, creating a new relationship 'cooperate_invest_supply':
cypher_query = """
MATCH (a:company)-[r1:compete]-(b:company)
MATCH (a)-[r2]-(b)
WHERE type(r2) IN ['cooperate', 'invest', 'supply']
WITH a, b, COLLECT(r2) AS relationships
FOREACH (rel IN relationships | CREATE (a)-[:cooperate_invest_supply]->(b) DELETE rel)
RETURN a, b;
"""

# Execute the Cypher query
result = execute_query(cypher_query)


C:\Users\cindy\AppData\Local\Temp\ipykernel_20504\386232654.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


In [13]:
## Find the company with the most 'cooperate' relationships and return the company and its 'cooperate' relationships:
cypher_query = """
MATCH (node:company)-[r:cooperate]->(otherNode)
WITH node, COUNT(r) AS incomingCount, COLLECT(r) AS relationships
ORDER BY incomingCount DESC
LIMIT 1
RETURN node, relationships;
"""

# Execute the Cypher query
result = execute_query(cypher_query)


In [ ]:
## Retrieve information for a specific company with code "300750.SZ", company_name "宁德时代", and ID 743:
cypher_query = """
MATCH (c:company {code: "300750.SZ", company_name: "宁德时代", ID: 743})-[r]-(connectedNode)
RETURN c, r, connectedNode;
"""

# Execute the Cypher query
result = execute_query(cypher_query)


In [ ]:
## Find the top 5 suppliers based on the number of 'supply' relationships:
cypher_query = """
MATCH (supplier:company)-[r:supply]->(customer:company)
WITH supplier, COUNT(r) AS outgoingSupplyCount
ORDER BY outgoingSupplyCount DESC
LIMIT 5
RETURN supplier, outgoingSupplyCount;
"""

# Execute the Cypher query
result = execute_query(cypher_query)


In [ ]:
## Find companies involved in disputes with more than 1 dispute relationship, and return the top 5:
cypher_query = """
MATCH (entity:company)-[r:dispute]->()
WITH entity, COUNT(r) AS disputeCount
WHERE disputeCount > 1
RETURN entity, disputeCount
ORDER BY disputeCount DESC
LIMIT 5;
"""

# Execute the Cypher query
result = execute_query(cypher_query)


In [ ]:
driver.close()


# Question 4: Knowledge-Driven Financial Analysis


WE will first make a database that list out the relationships between companies by concatenating all data. We separate two implicit effects of sentiment ( compete and dispute), and the other four together to remove redundancy.

### 4.1 association database curation

In [4]:
# Drop the 'time' column from each dataframe
df_compete = df_compete.drop('time', axis=1)
df_cooperate = df_cooperate.drop('time', axis=1)
df_dispute = df_dispute.drop('time', axis=1)
df_same_industry = df_same_industry.drop('time', axis=1)
# Concatenate 'df_compete' and 'df_dispute' together
df_oppo = pd.concat([df_compete, df_dispute])

# Concatenate the other four dataframes together
df_tgt = pd.concat([df_cooperate, df_invest, df_same_industry, df_supply])

to separate into two case, of opposite effect and unidirectional effect. 
Here we reverse the columns to record the bidirectional effect upon each other. Te

In [74]:
print(df_oppo.shape)
print(df_tgt.shape)

(464, 3)
(11202, 3)


In [6]:
import pandas as pd

# Select only the relevant columns
df_oppo_bidir = df_oppo[[':START_ID', ':END_ID']].copy()

# Create a copy with reversed start and end ids
df_reverse = df_oppo[[':END_ID', ':START_ID']].copy()
df_reverse.columns = [':START_ID', ':END_ID']

# Concatenate the original and reversed dataframes
df_oppo_bidir = pd.concat([df_oppo_bidir, df_reverse])


# Display the bidirectional dataframe shape
print(df_oppo_bidir.shape)


(928, 2)


In [7]:
# Create an empty dictionary to store relationships
relation_oppo_dict = {}

# Iterate through the dataframe and populate the dictionary
for index, row in df_oppo_bidir.iterrows():
    start_id = row[':START_ID']
    end_id = row[':END_ID']

    # Add start_id to the dictionary if not present
    if start_id not in relation_oppo_dict:
        relation_oppo_dict[start_id] = []

    # Add end_id to the list of related companies for the start_id
    relation_oppo_dict[start_id].append(end_id)

# Display the number of keys in the relationships dictionary
print("Number of keys:", len(relation_oppo_dict))


Number of keys: 507


THerefore, ther are 507 companies involved in compete / dispute relationships.
Now we do the same treatment on the ones with syngestic effect. 

In [8]:
# Select only the relevant columns
df_tgt_bidir = df_tgt[[':START_ID', ':END_ID']].copy()

# Create a copy with reversed start and end ids
df_reverse = df_tgt[[':END_ID', ':START_ID']].copy()
df_reverse.columns = [':START_ID', ':END_ID']

# Concatenate the original and reversed dataframes
df_tgt_bidir = pd.concat([df_tgt_bidir, df_reverse])

# Drop duplicate rows (keeping only distinct pairs of start and end ids)
df_tgt_bidir = df_tgt_bidir.drop_duplicates(subset=[':START_ID', ':END_ID'])

# Display the bidirectional dataframe shape
print(df_tgt_bidir.shape)


(17407, 2)


In [9]:
# Assuming your dataframe is named 'df'
# Create an empty dictionary to store relationships
relation_tgt_dict = {}

# Iterate through the dataframe and populate the dictionary
for index, row in df_tgt_bidir.iterrows():
    start_id = row[':START_ID']
    end_id = row[':END_ID']

    # Add start_id to the dictionary if not present
    if start_id not in relation_tgt_dict:
        relation_tgt_dict[start_id] = []

    # Add end_id to the list of related companies for the start_id
    relation_tgt_dict[start_id].append(end_id)

# Display the number of keys in the relationships dictionary
print("Number of keys:", len(relation_tgt_dict))


Number of keys: 3448


At most 3448 + 505 = 3955 companies, out of 3974 companies, have at least established 1 relationship with another company.


In [13]:
# Rename the dictionary
name_oppo_dict = {df_nodes.at[k, 'company_name']: [df_nodes.at[v, 'company_name'] for v in vs] for k, vs in relation_oppo_dict.items()}
name_oppo_dict = {key: list(set(values)) for key, values in name_oppo_dict.items()}

# Print the first 10 values of name_oppo_dict
for key, value in list(name_oppo_dict.items())[:10]:
    print(f'{key}: {value}')

中兴通讯: ['杰瑞股份', '中国电信']
中国平安: ['春光科技', '华夏幸福', '贵州茅台', '中国银行']
分众传媒: ['光线传媒', '中国联通']
贵州茅台: ['宁德时代', '山西汾酒', '中国平安', '贵绳股份']
宁德时代: ['永福股份', '贵州茅台', '易事特', '赣锋锂业', '智飞生物', '东方精工', '众泰汽车', '华自科技', '温氏股份', '沧州明珠', '先惠技术']
创新医疗: ['光大银行', '药明康德']
江铃汽车: ['中国神华']
赣锋锂业: ['宁德时代', '江特电机']
通威股份: ['迈为股份']
华创阳安: ['京东方']


In [14]:
name_tgt_dict = {df_nodes.at[k, 'company_name']: [df_nodes.at[v, 'company_name'] for v in vs] for k, vs in relation_tgt_dict.items()}
# make it distinct
name_tgt_dict = {key: list(set(values)) for key, values in name_tgt_dict.items()}

# Print the first 10 values of relation_tgt_dict
for key, value in list(name_tgt_dict.items())[:10]:
    print(f'{key}: {value}')

红宝丽: ['航锦科技', '滨化股份']
浙富控股: ['东方电气', '华凯创意', '二三四五', '长江电力', '葛洲坝', '汇通能源']
中兴通讯: ['长盈精密', '平安银行', '宜通世纪', '光迅科技', '国电南自', '移为通信', '海康威视', '中国石油', '容知日新', '辰安科技', '烽火通信', '宁德时代', '中国平安', '中国电信', '光大银行', '工业富联', '闻泰科技', '劲拓股份', '深圳华强', '鹏辉能源', '三一重工', '恩捷股份', '天迈科技', '新松机器人', '中国长城', '中信银行', '中国卫通', '寒武纪', '广电网络', '中国银行', '上汽集团', '天津港', '高新兴', '特发信息', '万马科技', '黑芝麻', '万马股份', '紫光股份', '格力电器', '千方科技', '比亚迪', '中光防雷', '智慧能源', '华泰证券', '美亚光电', '金智科技', '博创科技', '光华科技', '同有科技', '长安汽车', '国泰君安', '三七互娱', '苏交科', '北京银行', '顺络电子', '格尔软件', '民德电子', '中信证券', '春兴精工', '宣亚国际', '世纪华通', '聚飞光电', '中国联通', '佳都科技', '大中矿业', '博威合金', '上海银行', '中集集团', '民生银行', '中国石化', '浙大网新', '东风汽车', '许继电气', '兴民智通', '安科瑞', '奇安信', '建设银行', '国新文化', '紫金矿业', '赣锋锂业', '大唐电信', '梦网集团', '崇达技术', '温氏股份', '光峰科技', '贵州茅台', '亚光科技', '京东方', '新凤鸣', '天神娱乐', '创意信息', '国金证券', '交通银行', '招商证券']
潜能恒信: ['中国石油', '石化机械', '广电运通', '恒泰艾普']
小康股份: ['宁德时代', '工商银行', '飞龙股份', '东风汽车', '安徽合力', '中信建投', '比亚迪']
仁东控股: ['济民制药', '兴业银行', '永鼎股份', '姚记扑克']
卧龙地产: ['浙江龙盛']
江苏阳光: ['合盛硅业', '上机数

In [15]:
import math
import numpy as np

# Check for NaN values in relation_oppo_dict
nan_count_oppo = sum(math.isnan(val) if isinstance(val, (float, int)) else np.isnan(val).any() for val in relation_oppo_dict.values())
nan_count_tgt = sum(math.isnan(val) if isinstance(val, (float, int)) else np.isnan(val).any() for val in relation_tgt_dict.values())

# Print the counts
print(f"Number of NaN values in relation_oppo_dict: {nan_count_oppo}")
print(f"Number of NaN values in relation_tgt_dict: {nan_count_tgt}")
import math

def count_nan(dictionary):
    nan_count = 0
    for values in dictionary.values():
        for value in values:
            if isinstance(value, float) and math.isnan(value):
                nan_count += 1
    return nan_count

# Assuming name_oppo_dict and name_tgt_dict are your dictionaries
nan_count_oppo = count_nan(name_oppo_dict)
nan_count_tgt = count_nan(name_tgt_dict)

print(f'NaN count in name_oppo_dict: {nan_count_oppo}')
print(f'NaN count in name_tgt_dict: {nan_count_tgt}')


Number of NaN values in relation_oppo_dict: 0
Number of NaN values in relation_tgt_dict: 0
NaN count in name_oppo_dict: 0
NaN count in name_tgt_dict: 0


In [16]:
import json
# Save relation_oppo_dict to a file
with open('others/checkpoints/name_oppo_dict.json', 'w') as oppo_file:
    json.dump(name_oppo_dict, oppo_file, indent=2)

# Save relation_tgt_dict to a file
with open('others/checkpoints/name_tgt_dict.json', 'w') as tgt_file:
    json.dump(name_tgt_dict, tgt_file, indent=2)


### 4.2 matching 

We use the curated database name_tgt_dict and name_oppo_dict for implicit association.

In [22]:
df = pd.read_excel('C:/Users/cindy/OneDrive - HKUST (Guangzhou)/Mining/assignment/dsaa5002_project/5002Project_50015720_/Task1.xlsx')
print(df.head(2))

   NewsID                                        NewsContent Explicit_Company  \
0       1  　　本报记者 田雨 李京华    　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...             建设银行   
1       2  　　中国农业银行信用卡中心由北京搬到上海了！  　　农行行长杨明生日前在信用卡中心揭牌仪式上...             农业银行   

   label  
0      0  
1      1  


In [23]:
df['Implicit_Positive_Company'] = ''
df['Implicit_Negative_Company'] = ''

# Update the columns based on label values
for index, row in df.iterrows():
    explicit_companies = row['Explicit_Company'].split(',') if pd.notnull(row['Explicit_Company']) else []
    label = row['label']

    implicit_positive_companies = []
    implicit_negative_companies = []

    for company in explicit_companies:
        implicit_positive_companies.extend(name_oppo_dict.get(company, []))
        implicit_negative_companies.extend(name_tgt_dict.get(company, []))

    if label == 0:
        df.at[index, 'Implicit_Positive_Company'] = ', '.join(implicit_positive_companies)
        df.at[index, 'Implicit_Negative_Company'] = ', '.join(implicit_negative_companies)
    elif label == 1:
        df.at[index, 'Implicit_Positive_Company'] = ', '.join(implicit_negative_companies)
        df.at[index, 'Implicit_Negative_Company'] = ', '.join(implicit_positive_companies)

# Print the first 3 rows of the updated DataFrame
print(df.head(3))

   NewsID                                        NewsContent Explicit_Company  \
0       1  　　本报记者 田雨 李京华    　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...             建设银行   
1       2  　　中国农业银行信用卡中心由北京搬到上海了！  　　农行行长杨明生日前在信用卡中心揭牌仪式上...             农业银行   
2       3  　　在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...        外运发展,中国国航   

   label                          Implicit_Positive_Company  \
0      0                                          任子行, 捷捷微电   
1      1  东方财富, 神州数码, 山东矿机, 工商银行, 成都银行, 金地集团, 中信建投, 中科曙光...   
2      1  中国外运, 南方航空, 工商银行, 山航B, 中国卫通, 吉祥航空, 兴业证券, 寒武纪, ...   

                           Implicit_Negative_Company  
0  航天信息, 怡亚通, 工商银行, 聚龙股份, 平安银行, 晨鸣纸业, 中金公司, 民生银行,...  
1                                               ST云维  
2                                                     


In [25]:
#submission format for empty input
df['Implicit_Positive_Company'].replace('', 'None', inplace=True)
df['Implicit_Negative_Company'].replace('', 'None', inplace=True)

df.to_excel('Task2.xlsx', index=False)

In [26]:
df

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,建设银行,0,"任子行, 捷捷微电","航天信息, 怡亚通, 工商银行, 聚龙股份, 平安银行, 晨鸣纸业, 中金公司, 民生银行,..."
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,农业银行,1,"东方财富, 神州数码, 山东矿机, 工商银行, 成都银行, 金地集团, 中信建投, 中科曙光...",ST云维
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,"外运发展,中国国航",1,"中国外运, 南方航空, 工商银行, 山航B, 中国卫通, 吉祥航空, 兴业证券, 寒武纪, ...",None
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,胜利股份,1,"新疆浩源, 特锐德",None
4,8,由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致...,冠农股份,1,"藏格控股, 西部黄金, 南宁百货, 富邦股份",None
...,...,...,...,...,...,...
473680,1037031,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,亿华通,1,"东旭光电, 中国船舶, 飞龙股份, 东风汽车, 福田汽车, 百奥泰, 宝泰隆, 仕佳光子",None
473681,1037032,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,"中泰证券,依米康",1,"东北证券, 智飞生物, 五粮液, 史丹利, 中信建投, 国泰君安, 中金公司, 长江证券, ...","华谊嘉信, 西水股份"
473682,1037033,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,"中核科技,天风证券",1,"中国核电, 东北证券, 恒生电子, 浙商证券, 中信建投, 迈瑞医疗, 国泰君安, 中金公司...","三特索道, 中源家居, 吉翔股份"
473683,1037034,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,海特生物,1,"药明康德, 海尔生物",None
